In [63]:
import random

In [64]:
class Plant:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.grow_chance = 0.5

    def can_grow(self, time_of_day):
        return False

    def adapt(self, world):
        plant_density = world.plant_count / (world.width * world.height)
        self.grow_chance = max(0.1, 0.5 * (1 - plant_density))

    def grow(self, world, time_of_day):
        self.adapt(world)
        if not self.can_grow(time_of_day):
            return []
        neighbors = [(self.x+dx, self.y+dy) for dx, dy in [(-1,0), (1,0), (0,-1), (0,1)]]
        valid_neighbors = []
        for nx, ny in neighbors:
            if 0 <= nx < world.width and 0 <= ny < world.height and not world.grid[nx][ny]:
                valid_neighbors.append((nx, ny))
        if valid_neighbors and random.random() < self.grow_chance:
            nx, ny = random.choice(valid_neighbors)
            return [self.__class__(nx, ny)]
        return []

In [65]:
class Time:
    def __init__(self):
        self.cycle = 0
        self.times_of_day = ["morning", "day", "evening", "night"]
        
    def get_time(self):
        return self.times_of_day[self.cycle % 4]
    
    def tick(self):
        self.cycle += 1

In [66]:
class Dandelion(Plant):
    def can_grow(self, time_of_day):
        return time_of_day == "day"

class Amanita(Plant):
    def can_grow(self, time_of_day):
        return time_of_day == "night"

class Orange(Plant):
    def can_grow(self, time_of_day):
        return time_of_day in ["morning", "evening"]

In [67]:
class Animal:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.hunger = 0
        self.move_chance = 0.5
        self.attack_chance = 0.3

    def adapt(self, world):
        food_density = world.plant_count / (world.width * world.height)
        enemy_count = sum(1 for entity in world.entities if isinstance(entity, Animal) and not isinstance(entity, self.__class__))
        enemy_density = enemy_count / (world.width * world.height)
        self.move_chance = min(0.9, 0.5 + (0.5 - food_density))
        self.attack_chance = min(0.5, 0.3 + enemy_density)

    def act(self, world, time_of_day):
        pass

    def move(self, world):
        neighbors = [(self.x+dx, self.y+dy) for dx, dy in [(-1,0), (1,0), (0,-1), (0,1)]]
        valid_neighbors = [(nx, ny) for nx, ny in neighbors if 0 <= nx < world.width and 0 <= ny < world.height and not world.grid[nx][ny]]
        if valid_neighbors and random.random() < self.move_chance:
            world.grid[self.x][self.y] = None
            self.x, self.y = random.choice(valid_neighbors)
            world.grid[self.x][self.y] = self

In [68]:
class Baboin(Animal):
    def act(self, world, time_of_day):
        self.adapt(world)
        if time_of_day == "night":
            return
        self.hunger += 1
        if self.hunger > 10:
            world.remove_entity(self)
            return
        eat_chance = 0.8 if time_of_day == "morning" else 0.2 if time_of_day == "evening" else 0.5
        if random.random() < eat_chance:
            for dx, dy in [(-1,0), (1,0), (0,-1), (0,1)]:
                nx, ny = self.x + dx, self.y + dy
                if 0 <= nx < world.width and 0 <= ny < world.height:
                    entity = world.grid[nx][ny]
                    if isinstance(entity, Dandelion):
                        world.remove_entity(entity)
                        self.hunger = max(0, self.hunger - 2)
                        break
        if self.hunger > 5 and random.random() < self.attack_chance:
            for dx, dy in [(-1,0), (1,0), (0,-1), (0,1)]:
                nx, ny = self.x + dx, self.y + dy
                if 0 <= nx < world.width and 0 <= ny < world.height:
                    entity = world.grid[nx][ny]
                    if isinstance(entity, Baboin):
                        world.remove_entity(entity)
                        break
        self.move(world)

class Bear(Animal):
    def act(self, world, time_of_day):
        self.adapt(world)
        if time_of_day in ["day", "night"]:
            return
        self.hunger += 1
        if self.hunger > 10:
            world.remove_entity(self)
            return
        if random.random() < self.move_chance:
            self.move(world)
        for dx, dy in [(-1,0), (1,0), (0,-1), (0,1)]:
            nx, ny = self.x + dx, self.y + dy
            if 0 <= nx < world.width and 0 <= ny < world.height:
                entity = world.grid[nx][ny]
                if isinstance(entity, (Orange, Amanita, Baboin)):
                    world.remove_entity(entity)
                    self.hunger = max(0, self.hunger - 3)
                    break

In [69]:
class World:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.grid = [[None for _ in range(height)] for _ in range(width)]
        self.entities = []
        self.plant_count = 0

    def add_entity(self, entity):
        if not self.grid[entity.x][entity.y]:
            self.grid[entity.x][entity.y] = entity
            self.entities.append(entity)
            if isinstance(entity, Plant):
                self.plant_count += 1

    def remove_entity(self, entity):
        if entity in self.entities:
            self.grid[entity.x][entity.y] = None
            self.entities.remove(entity)
            if isinstance(entity, Plant):
                self.plant_count -= 1

    def step(self, time_of_day):
        new_entities = []
        for entity in self.entities[:] if self.entities else []:
            if isinstance(entity, Plant):
                new_plants = entity.grow(self, time_of_day)
                new_entities.extend(new_plants)
            elif isinstance(entity, Animal):
                entity.act(self, time_of_day)
        for entity in new_entities:
            self.add_entity(entity)

    def display(self):
        symbols = {Dandelion: 'О', Amanita: 'М', Orange: 'А', Baboin: 'B', Bear: 'R', None: '.'}
        for i in range(self.width):
            row = [symbols[type(self.grid[i][j])] if self.grid[i][j] else '.' for j in range(self.height)]
            print(' '.join(row))
        print()

In [70]:
def main():
    world = World(10, 10)
    time = Time()

    def place_entity(entity_class):
        while True:
            x, y = random.randint(0, 9), random.randint(0, 9)
            if not world.grid[x][y]:
                world.add_entity(entity_class(x, y))
                break

    for _ in range(10):
        place_entity(Dandelion)
        place_entity(Amanita)
        place_entity(Orange)
    for _ in range(5):
        place_entity(Baboin)
        place_entity(Bear)

    for step in range(20):
        time_of_day = time.get_time()
        print(f"Шаг {step + 1}, Время: {time_of_day}")
        world.step(time_of_day)
        world.display()
        time.tick()

if __name__ == "__main__":
    main()

Шаг 1, Время: morning
. М . . . . О . . О
О B О . А . М М . .
. B . О . . . . . .
О . О R . . О . . .
. О . R М . . . А .
М . . . А А А А А .
. . М B . . B . . .
. А А А . . . . . .
М . . . . . М . . R
А . R . . B . R . .

Шаг 2, Время: day
. М О . . . О О . О
О . О . А . М М . О
B B О О О . . . . .
О . О R . . О . . .
О О . R М . О . А .
М . . B А А А А А .
. . М . . B . . . .
. А А А . . . . . .
М . . . . . М . . R
А . R . . B . R . .

Шаг 3, Время: evening
. М О . . . О О . О
О . О . А . М М . О
B B . О О . . . . .
О . О R . . О . . .
О О . R М . О . А .
М . . . А А А А А .
. . М . . . B . . .
. А А А . . . . . .
М . . . . B М . . R
А . R . . . . R . .

Шаг 4, Время: night
. М О . . . О О . О
О . О . А . М М . О
B B . О О . М . . .
О . О R М . О . . .
О О . R М . О . А .
М . . . А А А А А .
. . М . . . B . . .
. А А А . . . . . .
М . . . . B М . . R
А . R . . . . R . .

Шаг 5, Время: morning
. М О . . . О О . О
B . О А А . М М . О
. . B О О . М . . .
О . О R . . О . . .
О О . R . . 